# Notebook 4 : P vs NP et Complexité Algorithmique

**Auteur** : Manus AI  
**Date** : 12 Octobre 2025

Ce notebook explore les classes de complexité, les problèmes NP-complets, et les stratégies pour y faire face.

## Table des Matières
1. Classes de Complexité P et NP
2. Réductions Polynomiales
3. Problèmes NP-Complets Classiques
4. Algorithmes d'Approximation
5. Heuristiques et Métaheuristiques

In [ ]:
!pip install numpy matplotlib plotly networkx scipy -q

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
from itertools import combinations, permutations
import time
from scipy.optimize import linear_sum_assignment

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Visualiser la Croissance des Fonctions de Complexité

In [ ]:
# Comparer différentes complexités
n = np.arange(1, 25)

# Différentes fonctions de complexité
O_1 = np.ones_like(n)
O_log_n = np.log2(n)
O_n = n
O_n_log_n = n * np.log2(n)
O_n2 = n**2
O_n3 = n**3
O_2n = 2**n
O_n_fact = np.array([np.math.factorial(min(i, 20)) for i in n])  # Limiter pour éviter overflow

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Graphique linéaire
ax1.plot(n, O_1, 'o-', label='O(1)', linewidth=2)
ax1.plot(n, O_log_n, 's-', label='O(log n)', linewidth=2)
ax1.plot(n, O_n, '^-', label='O(n)', linewidth=2)
ax1.plot(n, O_n_log_n, 'v-', label='O(n log n)', linewidth=2)
ax1.plot(n, O_n2, 'd-', label='O(n²)', linewidth=2)
ax1.plot(n[:15], O_2n[:15], 'p-', label='O(2ⁿ)', linewidth=2)

ax1.set_xlabel('Taille de l\'entrée (n)', fontsize=12)
ax1.set_ylabel('Nombre d\'opérations', fontsize=12)
ax1.set_title('Croissance des Fonctions de Complexité (échelle linéaire)', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1000)

# Graphique logarithmique
ax2.semilogy(n, O_1, 'o-', label='O(1)', linewidth=2)
ax2.semilogy(n, O_log_n, 's-', label='O(log n)', linewidth=2)
ax2.semilogy(n, O_n, '^-', label='O(n)', linewidth=2)
ax2.semilogy(n, O_n_log_n, 'v-', label='O(n log n)', linewidth=2)
ax2.semilogy(n, O_n2, 'd-', label='O(n²)', linewidth=2)
ax2.semilogy(n, O_n3, '<-', label='O(n³)', linewidth=2)
ax2.semilogy(n[:20], O_2n[:20], 'p-', label='O(2ⁿ)', linewidth=2)
ax2.semilogy(n[:12], O_n_fact[:12], '*-', label='O(n!)', linewidth=2, markersize=10)

ax2.set_xlabel('Taille de l\'entrée (n)', fontsize=12)
ax2.set_ylabel('Nombre d\'opérations (échelle log)', fontsize=12)
ax2.set_title('Croissance des Fonctions de Complexité (échelle logarithmique)', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("Observation clé:")
print("- Les algorithmes polynomiaux (O(n), O(n²), O(n³)) sont considérés 'efficaces'")
print("- Les algorithmes exponentiels (O(2ⁿ), O(n!)) deviennent rapidement impraticables")
print("- Pour n=20: O(n²) = 400, mais O(2ⁿ) ≈ 1 million !")
print("- Pour n=30: O(n²) = 900, mais O(2ⁿ) ≈ 1 milliard !")

## 2. Problème SAT : Le Premier Problème NP-Complet

Le problème de satisfiabilité booléenne (SAT) : étant donnée une formule logique, existe-t-il une assignation de valeurs qui la rend vraie ?

In [ ]:
def evaluate_clause(clause, assignment):
    """Évalue une clause (disjonction de littéraux)"""
    for literal in clause:
        var = abs(literal)
        value = assignment.get(var, False)
        # Si le littéral est négatif, inverser la valeur
        if literal < 0:
            value = not value
        if value:
            return True
    return False

def evaluate_cnf(clauses, assignment):
    """Évalue une formule en forme normale conjonctive (CNF)"""
    return all(evaluate_clause(clause, assignment) for clause in clauses)

def brute_force_sat(clauses, n_vars):
    """Résout SAT par force brute (exponentiel!)"""
    # Essayer toutes les 2^n assignations possibles
    for i in range(2**n_vars):
        # Convertir i en assignation binaire
        assignment = {}
        for var in range(1, n_vars + 1):
            assignment[var] = bool((i >> (var - 1)) & 1)
        
        # Vérifier si cette assignation satisfait la formule
        if evaluate_cnf(clauses, assignment):
            return True, assignment
    
    return False, None

# Exemple de formule 3-SAT
# (x1 ∨ ¬x2 ∨ x3) ∧ (¬x1 ∨ x2 ∨ x3) ∧ (x1 ∨ x2 ∨ ¬x3)
clauses = [
    [1, -2, 3],   # x1 ∨ ¬x2 ∨ x3
    [-1, 2, 3],   # ¬x1 ∨ x2 ∨ x3
    [1, 2, -3]    # x1 ∨ x2 ∨ ¬x3
]
n_vars = 3

print("Formule 3-SAT:")
print("(x₁ ∨ ¬x₂ ∨ x₃) ∧ (¬x₁ ∨ x₂ ∨ x₃) ∧ (x₁ ∨ x₂ ∨ ¬x₃)")
print()

start_time = time.time()
satisfiable, solution = brute_force_sat(clauses, n_vars)
end_time = time.time()

print(f"Satisfiable ? {satisfiable}")
if satisfiable:
    print(f"Solution trouvée: {solution}")
    print(f"Vérification:")
    for i, clause in enumerate(clauses, 1):
        result = evaluate_clause(clause, solution)
        print(f"  Clause {i}: {result}")
print(f"Temps d'exécution: {end_time - start_time:.6f} secondes")
print()

# Mesurer le temps pour différentes tailles
print("Temps d'exécution pour différentes tailles (force brute):")
sizes = [5, 10, 15, 20]
times = []

for n in sizes:
    # Créer une formule aléatoire
    np.random.seed(42)
    random_clauses = []
    for _ in range(3*n):  # 3n clauses
        clause = []
        for _ in range(3):  # 3 littéraux par clause
            var = np.random.randint(1, n+1)
            sign = np.random.choice([-1, 1])
            clause.append(sign * var)
        random_clauses.append(clause)
    
    start = time.time()
    # Limiter le nombre d'assignations testées pour les grandes tailles
    max_iter = min(2**n, 100000)
    for i in range(max_iter):
        assignment = {var: bool((i >> (var - 1)) & 1) for var in range(1, n + 1)}
        if evaluate_cnf(random_clauses, assignment):
            break
    end = time.time()
    
    elapsed = end - start
    times.append(elapsed)
    print(f"  n = {n}: {elapsed:.6f} secondes")

print()
print("Le temps croît exponentiellement avec n !")
print("C'est pourquoi SAT est NP-complet: facile à vérifier, difficile à résoudre.")

## 3. Problème du Voyageur de Commerce (TSP)

Trouver le plus court circuit visitant toutes les villes exactement une fois.

In [ ]:
def tsp_brute_force(distance_matrix):
    """Résout TSP par énumération exhaustive"""
    n = len(distance_matrix)
    cities = list(range(n))
    
    min_distance = float('inf')
    best_tour = None
    
    # Fixer la première ville (pour éviter les rotations)
    for perm in permutations(cities[1:]):
        tour = [0] + list(perm) + [0]  # Retour à la ville de départ
        distance = sum(distance_matrix[tour[i]][tour[i+1]] for i in range(n))
        
        if distance < min_distance:
            min_distance = distance
            best_tour = tour
    
    return best_tour, min_distance

def tsp_nearest_neighbor(distance_matrix, start=0):
    """Heuristique du plus proche voisin (glouton)"""
    n = len(distance_matrix)
    unvisited = set(range(n))
    tour = [start]
    unvisited.remove(start)
    
    current = start
    total_distance = 0
    
    while unvisited:
        # Trouver la ville non visitée la plus proche
        nearest = min(unvisited, key=lambda city: distance_matrix[current][city])
        total_distance += distance_matrix[current][nearest]
        tour.append(nearest)
        unvisited.remove(nearest)
        current = nearest
    
    # Retour au départ
    total_distance += distance_matrix[current][start]
    tour.append(start)
    
    return tour, total_distance

# Créer un petit problème TSP
np.random.seed(42)
n_cities = 6
cities_coords = np.random.rand(n_cities, 2) * 10

# Calculer la matrice de distances
distance_matrix = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(n_cities):
        distance_matrix[i][j] = np.linalg.norm(cities_coords[i] - cities_coords[j])

print(f"Problème TSP avec {n_cities} villes")
print()

# Solution optimale (force brute)
print("Solution optimale (énumération exhaustive):")
start_time = time.time()
optimal_tour, optimal_distance = tsp_brute_force(distance_matrix)
brute_force_time = time.time() - start_time
print(f"  Tour: {optimal_tour}")
print(f"  Distance: {optimal_distance:.2f}")
print(f"  Temps: {brute_force_time:.6f} secondes")
print()

# Heuristique du plus proche voisin
print("Heuristique du plus proche voisin:")
start_time = time.time()
greedy_tour, greedy_distance = tsp_nearest_neighbor(distance_matrix)
greedy_time = time.time() - start_time
print(f"  Tour: {greedy_tour}")
print(f"  Distance: {greedy_distance:.2f}")
print(f"  Temps: {greedy_time:.6f} secondes")
print(f"  Ratio d'approximation: {greedy_distance / optimal_distance:.2f}")
print()

# Visualiser les deux solutions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Solution optimale
ax1.scatter(cities_coords[:, 0], cities_coords[:, 1], s=200, c='red', zorder=5)
for i, (x, y) in enumerate(cities_coords):
    ax1.annotate(f'{i}', (x, y), fontsize=14, ha='center', va='center', color='white', weight='bold')

for i in range(len(optimal_tour) - 1):
    city1, city2 = optimal_tour[i], optimal_tour[i+1]
    ax1.plot([cities_coords[city1, 0], cities_coords[city2, 0]],
             [cities_coords[city1, 1], cities_coords[city2, 1]],
             'b-', linewidth=2, alpha=0.7)

ax1.set_title(f'Solution Optimale\nDistance: {optimal_distance:.2f}', fontsize=14)
ax1.set_xlabel('X', fontsize=12)
ax1.set_ylabel('Y', fontsize=12)
ax1.grid(True, alpha=0.3)

# Heuristique
ax2.scatter(cities_coords[:, 0], cities_coords[:, 1], s=200, c='red', zorder=5)
for i, (x, y) in enumerate(cities_coords):
    ax2.annotate(f'{i}', (x, y), fontsize=14, ha='center', va='center', color='white', weight='bold')

for i in range(len(greedy_tour) - 1):
    city1, city2 = greedy_tour[i], greedy_tour[i+1]
    ax2.plot([cities_coords[city1, 0], cities_coords[city2, 0]],
             [cities_coords[city1, 1], cities_coords[city2, 1]],
             'g-', linewidth=2, alpha=0.7)

ax2.set_title(f'Heuristique Plus Proche Voisin\nDistance: {greedy_distance:.2f}', fontsize=14)
ax2.set_xlabel('X', fontsize=12)
ax2.set_ylabel('Y', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observation:")
print("- L'heuristique est beaucoup plus rapide mais ne garantit pas l'optimalité")
print("- Pour n=10 villes: 9! = 362,880 permutations à tester")
print("- Pour n=20 villes: 19! ≈ 10^17 permutations (impossible!)")

## 4. Problème de la Clique Maximale

Trouver le plus grand sous-ensemble de sommets tous connectés entre eux dans un graphe.

In [ ]:
# Créer un graphe aléatoire
n_nodes = 10
p_edge = 0.4  # Probabilité d'une arête

np.random.seed(42)
G = nx.erdos_renyi_graph(n_nodes, p_edge, seed=42)

def is_clique(G, nodes):
    """Vérifie si un ensemble de nœuds forme une clique"""
    for i, node1 in enumerate(nodes):
        for node2 in nodes[i+1:]:
            if not G.has_edge(node1, node2):
                return False
    return True

def find_max_clique_brute_force(G):
    """Trouve la clique maximale par force brute"""
    nodes = list(G.nodes())
    max_clique = []
    
    # Essayer toutes les combinaisons de nœuds
    for size in range(len(nodes), 0, -1):
        for subset in combinations(nodes, size):
            if is_clique(G, subset):
                return list(subset)
    
    return max_clique

print(f"Graphe avec {n_nodes} nœuds et {G.number_of_edges()} arêtes")
print()

# Trouver la clique maximale
start_time = time.time()
max_clique = find_max_clique_brute_force(G)
end_time = time.time()

print(f"Clique maximale: {max_clique}")
print(f"Taille: {len(max_clique)}")
print(f"Temps: {end_time - start_time:.6f} secondes")
print()

# Visualiser le graphe avec la clique mise en évidence
plt.figure(figsize=(12, 10))

pos = nx.spring_layout(G, seed=42)

# Dessiner tous les nœuds
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)

# Mettre en évidence la clique maximale
nx.draw_networkx_nodes(G, pos, nodelist=max_clique, node_color='red', node_size=700)

# Dessiner toutes les arêtes
nx.draw_networkx_edges(G, pos, alpha=0.3)

# Mettre en évidence les arêtes de la clique
clique_edges = [(u, v) for u in max_clique for v in max_clique if u < v and G.has_edge(u, v)]
nx.draw_networkx_edges(G, pos, edgelist=clique_edges, edge_color='red', width=3)

# Labels
nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold')

plt.title(f'Problème de la Clique Maximale\nClique trouvée: {max_clique} (taille {len(max_clique)})', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

print("Le problème de la clique est NP-complet.")
print("Il faut potentiellement vérifier 2^n sous-ensembles de nœuds.")

## 5. Algorithme d'Approximation : Vertex Cover

Le problème Vertex Cover: trouver le plus petit ensemble de sommets touchant toutes les arêtes.

In [ ]:
def vertex_cover_approximation(G):
    """Algorithme d'approximation 2-facteur pour Vertex Cover"""
    cover = set()
    edges = list(G.edges())
    
    while edges:
        # Prendre une arête arbitraire
        u, v = edges[0]
        
        # Ajouter les deux extrémités à la couverture
        cover.add(u)
        cover.add(v)
        
        # Retirer toutes les arêtes incidentes à u ou v
        edges = [(a, b) for a, b in edges if a not in {u, v} and b not in {u, v}]
    
    return cover

# Créer un graphe
G_vc = nx.Graph()
G_vc.add_edges_from([(0,1), (0,2), (1,2), (1,3), (2,4), (3,4), (3,5), (4,5)])

print("Problème Vertex Cover")
print(f"Graphe avec {G_vc.number_of_nodes()} nœuds et {G_vc.number_of_edges()} arêtes")
print()

# Approximation
cover = vertex_cover_approximation(G_vc)
print(f"Couverture trouvée (approximation): {cover}")
print(f"Taille: {len(cover)}")
print()

# Vérifier que c'est bien une couverture
is_valid = all(u in cover or v in cover for u, v in G_vc.edges())
print(f"Couverture valide ? {is_valid}")
print()

# Visualiser
plt.figure(figsize=(10, 8))

pos = nx.spring_layout(G_vc, seed=42)

# Dessiner les nœuds
not_in_cover = [n for n in G_vc.nodes() if n not in cover]
nx.draw_networkx_nodes(G_vc, pos, nodelist=not_in_cover, node_color='lightblue', node_size=700)
nx.draw_networkx_nodes(G_vc, pos, nodelist=list(cover), node_color='red', node_size=900)

# Dessiner les arêtes
nx.draw_networkx_edges(G_vc, pos, width=2)

# Labels
nx.draw_networkx_labels(G_vc, pos, font_size=16, font_weight='bold', font_color='white')

plt.title(f'Vertex Cover (Approximation 2-facteur)\nCouverture: {cover}', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

print("Garantie théorique:")
print("- L'algorithme trouve une couverture de taille au plus 2 × OPT")
print("- C'est un algorithme d'approximation avec ratio 2")
print("- Il s'exécute en temps polynomial O(|E|)")

## 6. Diagramme des Classes de Complexité

In [ ]:
# Créer un diagramme conceptuel des classes de complexité
fig, ax = plt.subplots(figsize=(12, 10))

# Définir les rectangles pour les classes
from matplotlib.patches import Rectangle, FancyBboxPatch

# EXPTIME (le plus grand)
exptime = FancyBboxPatch((0.5, 0.5), 8, 7, boxstyle="round,pad=0.1", 
                         edgecolor='black', facecolor='lightgray', linewidth=3, alpha=0.3)
ax.add_patch(exptime)
ax.text(4.5, 7.2, 'EXPTIME', fontsize=20, weight='bold', ha='center')

# PSPACE
pspace = FancyBboxPatch((1, 1), 7, 5.5, boxstyle="round,pad=0.1",
                       edgecolor='blue', facecolor='lightblue', linewidth=2.5, alpha=0.4)
ax.add_patch(pspace)
ax.text(4.5, 6.2, 'PSPACE', fontsize=18, weight='bold', ha='center')

# NP
np_class = FancyBboxPatch((1.5, 1.5), 6, 4, boxstyle="round,pad=0.1",
                         edgecolor='red', facecolor='lightyellow', linewidth=2.5, alpha=0.5)
ax.add_patch(np_class)
ax.text(4.5, 5.2, 'NP', fontsize=18, weight='bold', ha='center', color='red')

# P
p_class = FancyBboxPatch((2, 2), 5, 2.5, boxstyle="round,pad=0.1",
                        edgecolor='green', facecolor='lightgreen', linewidth=2.5, alpha=0.6)
ax.add_patch(p_class)
ax.text(4.5, 3.8, 'P', fontsize=18, weight='bold', ha='center', color='darkgreen')

# Zone NP-Complet
npc = FancyBboxPatch((5.5, 2), 1.5, 2.8, boxstyle="round,pad=0.05",
                    edgecolor='darkred', facecolor='salmon', linewidth=2, alpha=0.7)
ax.add_patch(npc)
ax.text(6.25, 3.4, 'NP-\nComplet', fontsize=12, weight='bold', ha='center', va='center')

# Ajouter des exemples de problèmes
ax.text(4.5, 3.2, 'Tri\nRecherche\nPlus court chemin', 
        fontsize=10, ha='center', va='center', style='italic')
ax.text(3.2, 4.5, 'Factorisation\n(supposé)', 
        fontsize=9, ha='center', va='center', style='italic')
ax.text(6.25, 4.5, 'SAT\nTSP\nClique\nVertex Cover', 
        fontsize=9, ha='center', va='center', style='italic', color='darkred')

# Ajouter la question P vs NP
ax.annotate('P = NP ?', xy=(4.5, 2.5), xytext=(1, 0.3),
           fontsize=24, weight='bold', color='purple',
           arrowprops=dict(arrowstyle='->', lw=3, color='purple'),
           bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.8))

ax.set_xlim(0, 9)
ax.set_ylim(0, 8)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Hiérarchie des Classes de Complexité', fontsize=22, weight='bold', pad=20)

plt.tight_layout()
plt.show()

print("Hiérarchie connue: P ⊆ NP ⊆ PSPACE ⊆ EXPTIME")
print("On sait que P ≠ EXPTIME, donc au moins une inclusion est stricte.")
print("Mais on ne sait pas laquelle !")
print()
print("La question P vs NP est l'un des 7 Problèmes du Prix du Millénaire.")
print("Résoudre cette question vaut 1 million de dollars du Clay Mathematics Institute.")

## Conclusion

Ce notebook a exploré :

- Les **classes de complexité** P et NP
- Les **problèmes NP-complets** (SAT, TSP, Clique, Vertex Cover)
- La différence entre **résoudre** et **vérifier** un problème
- Les **algorithmes d'approximation** avec garanties de performance
- Les **heuristiques** pour obtenir des solutions pratiques

La théorie de la complexité nous aide à comprendre les limites fondamentales du calcul et à concevoir des stratégies pragmatiques face à des problèmes difficiles.